### Import Modules

In [43]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time
from importlib import reload

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [75]:
reload(dl)
reload(sf)
reload(fit)
reload(bs)

<module 'fitter.bootstrapper' from '../fitter/bootstrapper.py'>

### Specify fit parameters

In [76]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo', or 'nnnlo'
        'vol' : 10, # max 10
        'exclude' : [], # put LECs here
        
        # semi-nnlo corrections
        'include_alpha_s' : True,
        'include_log' : True, 
        'include_log2' : False,
        'include_sunset' : False
    }, 
    'fit_type' : 'xpt-ratio', #'ma', 'ma-ratio, 'xpt', or 'xpt-ratio'; also 'simultaneous', 'ma-old'
    'F2' : 'FpiFpi', #'FKFpi', 'FpiFpi', 'FKFK', 'F0F0' (HPQCD choice)
    'bias_correct' : True,
    'include_su2_isospin_corrrection' : False,
    
    'use_prior' : False,
    'use_bijnens_central_value' : True,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
    
    'make_plots' : False,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_results' : False,
}

In [77]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(fit_type=p_dict['fit_type'], F2=p_dict['F2'], include_FV=(p_dict['order']['vol']>0), 
                                 include_alphaS=p_dict['order']['include_log'], include_logSq=p_dict['order']['include_log2'])

# Make bootstrapper
bootstrapper = bs.bootstrapper(
    fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], 
    include_su2_isospin_corrrection=p_dict['include_su2_isospin_corrrection'], use_bijnens_central_value=p_dict['use_bijnens_central_value'],
    fit_type=p_dict['fit_type'], abbrs=p_dict['abbrs'], bias_correct=p_dict['bias_correct']
)

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename='fits/'+bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
else:
    pass #print(bootstrapper)
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info())
    
    
t1 = time.time()

print("\nTotal time (s): ", t1 - t0, "\n")

Using default prior.

Total time (s):  0.5966231822967529 



In [80]:
print(bootstrapper._make_empbayes_fit())

{'chiral': 1.0,'spacing': 1.0}
0.0(1.0) 0.0(1.0)
{'chiral': 1.05,'spacing': 1.0}
0.0(1.1) 0.0(1.0)
{'chiral': 1.0,'spacing': 1.05}
0.0(1.0) 0.0(1.1)
{'chiral': 1.0499999999999998,'spacing': 1.0499999999999998}
0.0(1.0) 0.0(1.0)
{'chiral': 1.0749999999999997,'spacing': 1.0749999999999997}
0.0(1.1) 0.0(1.1)
{'chiral': 1.0249999999999997,'spacing': 1.125}
0.0(1.0) 0.0(1.1)
{'chiral': 1.0999999999999996,'spacing': 1.1499999999999997}
0.0(1.1) 0.0(1.1)
{'chiral': 1.1499999999999995,'spacing': 1.1999999999999997}
0.0(1.1) 0.0(1.2)
{'chiral': 1.1999999999999995,'spacing': 1.1499999999999995}
0.0(1.2) 0.0(1.1)
{'chiral': 1.2874999999999992,'spacing': 1.1624999999999992}
0.0(1.3) 0.0(1.2)
{'chiral': 1.362499999999999,'spacing': 1.2874999999999992}
0.0(1.4) 0.0(1.3)
{'chiral': 1.5062499999999988,'spacing': 1.393749999999999}
0.0(1.5) 0.0(1.4)
{'chiral': 1.6437499999999985,'spacing': 1.3562499999999984}
0.0(1.6) 0.0(1.4)
{'chiral': 1.8906249999999982,'spacing': 1.4343749999999975}
0.0(1.9) 0.0(1.

In [78]:
print(bootstrapper)

Making fits...
Done!
100% complete
Time (s):  6.267195701599121
Compiling results...

Model: xpt-ratio_FpiFpi_nnlo_log_alphaS_FV_bijnens

Fitting to nnlo 
 with volume corrections O(10) 
 chained: False 
Fitted/[FLAG] values at physical point (including SU(2) isospin corrections: False):

F_K / F_pi = 1.1855(71) [1.1956(20)]   (delta_su2 = -0.00396(71))

  Parameter    Result[0] / Prior[1]    Parameter    Result[0] / Prior[1]
-----------------------------------  -----------------------------------
      A_a 0              -2.8 (2.8)        L_3 0           -0.00220 (95)
          1               0.0 (5.0)            1            -0.0031 (15)
      A_k 0               4.3 (1.4)        L_4 0           -0.00095 (19)
          1               0.0 (5.0)            1             0.0000 (50)
   A_loga 0              -0.4 (4.1)        L_5 0           -0.00147 (48)
          1               0.0 (5.0)            1             0.0000 (50)
      A_p 0              -1.3 (2.8)        L_6 0           

In [5]:
fit_data = {
    'mpi' : 135.,
    'mk' : 495.5,
    'Fpi' : 92.2,
    'FK' : 110.5
}
fit_data['lam2_chi'] = (4 *np.pi *fit_data['Fpi'])**2

fit_parameters = {
    'L_1' : 0.000372,
    'L_2' : 0.000493,
    'L_3' : -0.003070,
    'L_4' : 0.000089,
    'L_5' : 0.000377,
    'L_6' : 0.000011,
    'L_7' : -0.000340,
    'L_8' : 0.000294,
    'A_k' : -3.000000,
    'A_p' : 4.000000
}


bootstrapper.fk_fpi_fit_fcn(fit_data=fit_data, fit_parameters=fit_parameters, debug=True)

1 + nlo 1.2098507616726104
ct -0.08370923117181872
extra ct 0.009271430036091247
mu fix -0.0605200744906015
FF 0.012421152702167185
log 0.07901253947568393
logSq 0.028598430896321657
nnlo -0.014925752552156224
p {'L_1': 0.000372, 'L_2': 0.000493, 'L_3': -0.00307, 'L_4': 8.9e-05, 'L_5': 0.000377, 'L_6': 1.1e-05, 'L_7': -0.00034, 'L_8': 0.000294, 'A_k': -3.0, 'A_p': 4.0, 'mpi': 135.0, 'mk': 495.5, 'Fpi': 92.2, 'FK': 110.5, 'lam2_chi': 1342398.8460313035}


1.1949250091204542